In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy.stats.mstats import zscore

In [2]:
df_study_3 = pd.read_csv('../../Data/study4_olr_formatted.csv', delimiter=',')

In [3]:
#Get all got emotion valuation columns in separate arrays
cols = df_study_3.columns

eval_cols = [i for i in cols if len(i.split('Good'))>1]
actual_cols = [i for i in cols if len(i.split('Actual'))>1]
ideal_cols = [i for i in cols if len(i.split('Ideal'))>1]
hedonic_cols = [i for i in cols if len(i.split('Pleasant'))>1]

In [4]:
#emotion concepts for multiplte regression
emo_concepts_regression = ['Shame',
 'Fear',
 'Calm',
 'Jealousy',
 'Miserable',
 'Lust',
 'Disgust',
 'Guilt',
 'Pride',
 'Excitement',
 'Sadness',
 'Happy',
 'Love']
model_summaries = []

#run multiple regression predicting actual emotion experience from  emotion valuations
for i in range(len(emo_concepts_regression)):
    emotion = emo_concepts_regression[i]
    eval_col = emotion+ 'Good'
    ideal_col = emotion+'Ideal'
    hedonic_col = emotion + 'Pleasant'
    X = df_study_3[[eval_col,ideal_col,hedonic_col]]
    y = df_study_3[emotion+str('Actual')]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X) # make the predictions by the model

    # Print out the statistics
    model_summaries.append(model)
    

/Users/phoenix/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return getattr(obj, method)(*args, **kwds)


## Standardized Betas

In [5]:
model_summaries_z_scores = []
for i in range(len(emo_concepts_regression)):
    emotion = emo_concepts_regression[i]
    eval_col = emotion+str('Good')
    ideal_col = emotion+'Ideal'
    hedonic_col = emotion + 'Pleasant'
    X = zscore(df_study_3[[eval_col,ideal_col,hedonic_col]])
    y = zscore(df_study_3[emotion+str('Actual')])
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X) # make the predictions by the model

    # Print out the statistics
    model_summaries_z_scores.append(model)

## Save data

In [6]:
def create_df_of_betas(model_summaries):
    P_Val_Const = []
    P_Val_Good = []
    P_Val_Ideal = []
    P_Val_Pleasant = []
    B_Val_Const = []
    B_Val_Good = []
    B_Val_Ideal = []
    B_Val_Pleasant = []
    r_squared = []
    for i in range(len(emo_concepts_regression)):
        P_Val_Const.append(model_summaries[i].pvalues[0])
        P_Val_Good.append(model_summaries[i].pvalues[1])
        P_Val_Ideal.append(model_summaries[i].pvalues[2])
        P_Val_Pleasant.append(model_summaries[i].pvalues[3])
        B_Val_Const.append(model_summaries[i].params[0])
        B_Val_Good.append(model_summaries[i].params[1])
        B_Val_Ideal.append(model_summaries[i].params[2])
        B_Val_Pleasant.append(model_summaries[i].params[3])
        r_squared.append(model_summaries[i].rsquared)

    return pd.DataFrame({
        'emotion':emo_concepts_regression,
        'P_Val_Const':P_Val_Const,
        'P_Val_Good':P_Val_Good,
        'P_Val_Ideal':P_Val_Ideal,
        'P_Val_Pleasant':P_Val_Pleasant,
        'B_Val_Const':B_Val_Const,
        'B_Val_Good':B_Val_Good,
        'B_Val_Ideal':B_Val_Ideal,
        'B_Val_Pleasant':B_Val_Pleasant,
        'R_squared':r_squared
    })    

In [7]:
#create_df_of_betas(model_summaries_z_scores).to_excel('../../Data/Tables/RAW/Standardized_Betas_Multiple_Regression.xlsx')